###> Create the neural network

In [ ]:
import tensorflow as tf
from model import DEFAULT_MODEL_PARAMS, create_model

In [ ]:
model = create_model(DEFAULT_MODEL_PARAMS)
print("Created Model.")

###> Train the model

In [ ]:
from model import BATCH_SIZE, train
from process_flows import load_data

In [ ]:
x_train, y_train, x_test, y_test = load_data("tir")
print("Got the training and testing data")

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_dataset = val_dataset.batch(BATCH_SIZE)

In [ ]:
model = train(model, train_dataset, val_dataset)

###> Store the model

In [ ]:
from model import save

In [ ]:
save(model, 'tir')

###> Predict

In [ ]:
from process_flows import load_data
from model import predict

_, _, x_pred, y_truth = load_data("tir")

In [ ]:
props = predict(model, x_pred)

In [ ]:
def print_stats(props, y_truth):
    for threshold in [round(x*0.1, 1) for x in range(0,11)]:
      print(f'Threshold: {threshold}')
      y_pred = (tf.sigmoid(props) >= threshold).numpy().astype(float)

      tn, fp, fn, tp = metrics.confusion_matrix(y_truth, y_pred).ravel()

      tpr = tp/(tp+fn)
      fpr = fp/(fp+tn)
      bin_ac = (tp+tn)/(tn+tp+fp+fn)
      recall = tp / (tp + fn)
      precision = tp / (tp + fp)
      specificity = tn / (tn + fp)
      npv = tn / (tn + fn)
      f1 = 2 / ((1 / precision) + (1 / recall))
      p4 = 4 / ((1 / precision) + (1 / recall) + (1 / specificity) + (1 / npv))
      
      print(f'TPR: {np.round(tpr*100, 4)}%; FPR: {np.round(fpr*100, 4)}%; BinAcc: {np.round(bin_ac*100, 4)}%')     
      print(f'Recall: {np.round(recall*100, 4)}%; precision: {np.round(precision*100, 4)}%;\n'
            f'specificity: {np.round(specificity*100, 4)}%; NPV: {np.round(npv*100, 4)}%\n'
            f'f1-score: {np.round(f1*100, 4)}; p4: {np.round(p4*100, 4)}')

In [ ]:
def plot_roc(props, y_truth):
    scores = tf.sigmoid(props).numpy().astype(float)
    fpr, tpr, thresholds = metrics.roc_curve(y_truth, scores, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange',
             label=f'ROC curve (auc = {auc:.3f})')
    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic Curve')
    plt.legend()
    plt.show()
    return

In [ ]:
print_stats(props, y_truth)
plot_roc(props, y_truth)